# 校园智慧农场演示系统客户端


## 1.功能描述

- 定时（每隔1分钟）将数据上传到服务器；
- 根据土壤湿度的预设阈值，能够自行浇水；
- 实时响应控制指令，收到一次浇水指令，打开继电器（电磁阀）5秒钟。

In [ ]:

print(IOT_pubTopic)

## 2.代码说明

- SIoT地址为：192.168.3.8
- 光线传感器接A0，温湿度传感器接D4，继电器（电磁阀）模块接D2；
- 项目名：aifarm88（aifarm+编号），教师演示为88
- topicid名称

设备 | 引脚 | topicid
---|---|---
光线  | A0  | aifarm88/a0
环境温度  | D4  | aifarm88/d4
环境湿度  | D4  | aifarm88/d4
土壤湿度  | A1  | aifarm88/a1
继电器（电磁阀）  | D2  | aifarm88/d2


In [9]:
import time
from pinpong.board import Board,Pin,DHT11
Board("xugu").begin()
dht11 = DHT11(Pin(Pin.D4))
while True:
    c=[dht11.temp_c(),dht11.humidity()]
    print(c)
    time.sleep(1)

SyntaxError: invalid syntax (pymata4.py, line 330)

In [ ]:
import siot,time
from xugu import Pin
t1=60 # 上传数据的间隔时间，单位秒
t2=10 # 电磁阀打开的保持时间，单位秒
soil=1000 # 土壤湿度传感器数值阈值，建议大于500
SERVER = "127.0.0.1"        #MQTT服务器IP
CLIENT_ID = ""              #在SIoT上，CLIENT_ID可以留空
projectname='aifarm88' # 项目名称
pubTopiclist= ['a0','d4','a1','d2'] # 设备名称
IOT_pubTopic = [projectname +'/'+ x for x in pubTopiclist]
IOT_UserName ='scope'     #用户名
IOT_PassWord ='scope'     #密码

p0 = Pin("A0", Pin.ANALOG) # 光线传感器
p1 = Pin("A1", Pin.ANALOG) # 土壤湿度传感器
p2 = Pin(2, Pin.OUT) # 继电器（电磁阀）

ret = 0
def sub_cb(client, userdata, msg):
    global ret
    print("\nTopic:" + str(msg.topic) + " Message:" + str(msg.payload))
    if msg.payload==b'1':
        ret = 1
    if msg.payload==b'0':
        ret = 0

siot.init(CLIENT_ID, SERVER, user=IOT_UserName, password=IOT_PassWord)
siot.connect()
siot.subscribe(IOT_pubTopic[3], sub_cb)
siot.loop()

while True:
    p0_v = p0.read_analog()
    p1_v = p1.read_analog()
    if ret==1 or p1_v>soil:
        p2.write_digital(1)
        time.sleep(t2)
        p2.write_digital(0)
        ret=0
    else:
        siot.publish(IOT_pubTopic[0],"%d"%p0_v)
        siot.publish(IOT_pubTopic[2],"%d"%p1_v)
        time.sleep(t1)
        

## 3.其他说明

1）土壤传感器的使用

由于土壤传感器数值会受到入土深度、土壤松紧度的影响，只能检测到土壤的相对湿度，我们把湿度的范围分为三等分，分别表示干燥、湿润、非常潮湿。之前记入的两个数据为湿度区间。例如：空气中读数为570，水中读数为0，这样就可以分为（570,380],(380,190],(190,0]这三个区间分别代表干燥、湿润、非常潮湿。

2）环境温度和湿度的数据格式

考虑到环境温度和湿度是密切关联的，于是以列表的形式，记录在同一个topicid中，使用时要进行分割。